# 添加算子映射关系高级教程
`Linux` `Ascend` `GPU` `CPU` `模型迁移` `高级`

[![](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/_static/logo_source.png)](https://gitee.com/mindspore/mindinsight/blob/master/mindinsight/mindconverter/tutorial/add_operator_mapper_advanced_tutorial.ipynb)

## 概述

在确定ONNX算子到MindSpore算子的映射关系时，会遇到两者之间不存在相似实现或者参数差异过大难以直接转换的算子的问题。本文将在[初级教程](https://gitee.com/mindspore/mindinsight/blob/master/mindinsight/mindconverter/tutorial/add_operator_mapper_base_tutorial.ipynb)的基础上，以该类算子映射关系为例，来描述添加算子映射关系文件的方法。

## 环境准备

本案例需安装以下Python三方库：
```bash
pip install mindspore==1.2.0
pip install mindinsight==1.2.0
```

## 自定义添加算子映射脚本

以`onnx::AveragePool`算子为例进行演示。

分别查阅[ONNX算子API文档](https://github.com/onnx/onnx/blob/master/docs/Operators.md)和[MindSpore算子API文档](http://www.mindspore.cn/doc/api_python/zh-CN/master/index.html)，
找到与ONNX算子`onnx::AveragePool`功能相同或相近的MindSpore算子`mindspore.nn.AvgPool2d`。

|算子名|`onnx::AveragePool`|`mindspore.nn.AvgPool2d`|
|:----:|:----|:----|
|算法实现|`output_shape[i] = floor((input_shape[i]+pad_shape[i]-kernel_shape[i])/strides_shape[i])`<br>OR<br>`output_shape[i] = ceil((input_shape[i]+pad_shape[i]-kernel_shape[i])/strides_shape[i])` based on `ceil_mode`|`output_shape[i] = ceil((input_shape[i]-kernel_size[i]+1)/stride_shape[i])`<br>OR<br>`output_shape[i] = ceil(input_shape[i]/stride_shape[i])` based on `pad_mode`|
|参数|`auto_pad`: DEPRECATED<br>`ceil_mode`: optional<br>`count_include_pad`: optional<br>`kernel_shape`: optional<br>`pads`: optional<br>`strides`: optional|`kernel_size`: optional<br>`stride`: optional<br>`pad_mode`: optional<br>`data_format`: optional<br>|
|输入|`X`: required|`input`: required|
|输出|`Y`|`output`|

<br>
依据双方算子中参数（Attributes/Parameters）和输入（Inputs）进行ONNX到MindSpore的算子映射。

In [1]:
import math

import numpy as np

from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper
from mindinsight.mindconverter.graph_based_converter.constant import ExchangeMessageKeywords, TemplateKeywords


class PoolMapper(ONNXToMindSporeMapper):
    """Pool mapper."""

    @staticmethod
    def _operation_name_in_ms(*args, **kwargs):
        if kwargs['op_name'] == 'onnx::AveragePool':
            op_name = 'nn.AvgPool{}d'
        else:
            op_name = 'nn.MaxPool{}d'
        dim = len(kwargs['params']['strides'])
        return op_name.format(dim)

    @staticmethod
    def _convert_params(**kwargs):
        params = kwargs['params']
        transformed_params = dict()
        transformed_params["kernel_size"] = tuple(params['kernel_shape'])
        transformed_params["stride"] = tuple(params['strides'])

        return transformed_params

    @staticmethod
    def _convert_trained_weights(**kwargs):
        return dict()

    @staticmethod
    def _get_ms_opt_shape(**kwargs):
        """用于计算MindSpore算子在使用ONNX参数时，由`input_shape`得到的`output_shape`。"""
        params = kwargs['raw_params']
        input_shape = params['input_shape']
        kernel_shape = params['kernel_shape']
        strides = params['strides']
        dilations = params.get('dilations', (1, 1))
        ms_opt_shape = np.true_divide(np.subtract(np.array(input_shape[-len(kernel_shape):], dtype=np.float32),
                                                  ((np.array(kernel_shape, dtype=np.float32) - 1) *
                                                   np.array(dilations, dtype=np.float32) + 1)) + 1,
                                      np.array(strides, dtype=np.float32)).tolist()
        ms_opt_shape_ceil = tuple(math.ceil(ms_opt_shape_axis) for ms_opt_shape_axis in ms_opt_shape)
        return ms_opt_shape_ceil

    @staticmethod
    def _generate_snippet_template(**kwargs):
        """
        对于无法直接使用`_convert_params`方法进行参数映射的算子，重写此方法通过自定义的模板
        来生成算子在转换脚本中的定义（`init`）和调用（`construct`)。

        Args:
            operation (str): MindSpore中的对应算子名。
            converted_params (dict): 由`_convert_params`方法转换得到的MindSpore算子的参数。
            raw_params (dict): ONNX算子的参数(`raw_params`)，`input_shape`和`output_shape`。
        """

        op = kwargs.get("operation")
        args = kwargs.get("converted_params", dict())

        ms_opt_shape = PoolMapper._get_ms_opt_shape(**kwargs)
        tensor_opt_shape = kwargs['raw_params']['output_shape']
        tensor_ipt_shape = kwargs['raw_params']['input_shape']
        kernel_shape = kwargs['raw_params']['kernel_shape']
        dilations = kwargs['raw_params'].get('dilations', (1, 1))
        strides = kwargs['raw_params']['strides']

        if not op:
            raise ValueError("Can not get MindSpore operation name.")

        # 定义生成代码的模板。`init_xx`是在`init`中的算子定义，`construct_xx`是在`construct`中的算子调用，
        # 其中的`variable_slot`是替换用标签，会被后续的脚本生成模块填充。
        variable_slot = "var_0"
        init_template = f"self.{{{variable_slot}}} = {op}({', '.join(['%s={%s}' % (p, p) for p in args])})"
        construct_template = f"opt_{{{variable_slot}}} = self.{{{variable_slot}}}(opt_{{{variable_slot}}})"

        # 由于该算子在ONNX和MindSpore中的实现差异较大，为了保证转换结果的一致性，需要添加`mindspore.nn.Pad`算子，
        # 对输入进行处理之后，再传入算子中进行推理。
        # 该方法的输出依次为`Pad`算子定义，`Pad`算子调用和`Pad`算子的参数`paddings`。
        init_template_pad, construct_template_pad, paddings = \
            PoolMapper._generate_pad_init_and_construct(tensor_opt_shape, tensor_ipt_shape,
                                                        ms_opt_shape, variable_slot,
                                                        kernel_shape, dilations, strides)

        # 返回给后续模块的生成模板数据体，将按照列表顺序依次生成算子定义和算子调用，
        # `TemplateKeyWords.INIT.value`和`TemplateKeyWords.CONSTRUCT.value`分别表示`init`和`construct`。
        template = {
            variable_slot: {
                TemplateKeywords.INIT.value: [init_template_pad, init_template],
                TemplateKeywords.CONSTRUCT.value: [construct_template_pad, construct_template]
            }
        }

        # 新添加算子`Pad`的参数`paddings`也作为算子`Pool`的参数进行返回，使该参数也能正确的进行设置。
        args['paddings'] = paddings

        # 用于与后续模块进行信息交换。
        exchange_msg = {
            variable_slot: {
                ExchangeMessageKeywords.VariableScope.value.OPERATION.value: op,  # MindSpore算子名。
                ExchangeMessageKeywords.VariableScope.value.VARIABLE_NAME.value: None,  # 算子对应的变量名，由后续模块填写，此处为None。
                ExchangeMessageKeywords.VariableScope.value.OUTPUT_TYPE.value:
                    ExchangeMessageKeywords.VariableScope.value.TSR_TYPE.value,  # 算子输出的类型，`mindspore.Tensor`或者`Tuple<mindspore.Tensor>`。
                ExchangeMessageKeywords.VariableScope.value.INPUTS.value: [],  # 算子输入，由后续模块填写，此处为list()。
                ExchangeMessageKeywords.VariableScope.value.ARGS.value: args,  # 算子参数。
                ExchangeMessageKeywords.VariableScope.value.WEIGHTS.value: dict(),  # 算子的权重信息。
                ExchangeMessageKeywords.VariableScope.value.TRAINABLE_PARAMS.value: dict()  # 算子的可训练权重信息。由`_convert_trained_weights`方法返回。
            }
        }
        # 算子输出的变量名。若为多输出，则按照列表顺序依次生成。
        outputs_list = [f"opt_{{{variable_slot}}}"]
        # ONNX算子和MindSpore算子输出的对应顺序，主要用于保证多输出算子输出拓扑序的一致性。
        outputs_mapping = ((0, 0),)
        return template, exchange_msg, outputs_list, outputs_mapping

    @staticmethod
    def _generate_pad_init_and_construct(tensor_opt_shape, tensor_ipt_shape,
                                         ms_opt_shape, variable_slot, kernel_shape, dilations, strides):
        """
        生成`Pad`算子定义语句，`Pad`算子调用语句和计算参数`paddings`。

        Args:
            tensor_opt_shape (tuple): ONNX算子输出尺寸。
            tensor_ipt_shape (tuple): ONNX算子输入尺寸。
       ms_opt_shape (tuple): MindSpore算子输出尺寸。
            variable_slot (str): 用于后续模块进行替换的标识符。
            kernel_shape (Union[tuple, int]): ONNX算子参数`kernel_shape`。
            dilations (Union[tuple, int]): ONNX算子参数`dilations`。
            strides (Union[tuple, int]): ONNX算子参数`strides`。
        """

        onnx_opt_shape = tensor_opt_shape[-len(ms_opt_shape):]
        onnx_ipt_shape = tensor_ipt_shape[-len(ms_opt_shape):]

        if np.any(np.array(ms_opt_shape) > np.array(onnx_opt_shape)):
            raise ValueError(f"ms_opt_shape[{ms_opt_shape}] should be no larger than onnx_opt_shape[{onnx_opt_shape}].")

        if np.all(np.array(ms_opt_shape) == np.array(onnx_opt_shape)):
            shape_diff = np.zeros(len(ms_opt_shape)).astype(np.int).tolist()
        else:
            shape_diff = np.subtract((np.array(onnx_opt_shape) - 1) * np.array(strides),
                                     np.subtract(np.array(onnx_ipt_shape),
                                                 (np.array(kernel_shape) - 1) * np.array(dilations) + 1)).tolist()

        zero_pad_single = (0, 0)
        paddings = [zero_pad_single]
        num_zero_pads = len(tensor_opt_shape) - len(ms_opt_shape)
        for _ in range(num_zero_pads - 1):
            paddings.append(zero_pad_single)

        for axis_diff in shape_diff:
            paddings.append((int(axis_diff // 2), int(axis_diff // 2 + axis_diff % 2)))

        init_template_pad = f"self.pad_{{{variable_slot}}} = nn.Pad(paddings={{paddings}})"
        construct_template_pad = f"opt_{{{variable_slot}}} = self.pad_{{{variable_slot}}}" \
                                 f"({{{ExchangeMessageKeywords.VariableScope.value.INPUTS.value}}})"

        return init_template_pad, construct_template_pad, tuple(paddings)

将该Mapper脚本命名为`pool_mapper.py`，该命名方式需要和类名（`PoolMapper`）相对应。<br>
并放入 `mindinsight/mindconverter/graph_based_converter/mapper/impl/nn`目录下，该放置目录需要根据对应的MindSpore算子所在的层（`nn`/`ops`）来设置。<br>
最后修改 `mindinsight/mindconverter/graph_based_converter/mapper/onnx_to_ms.json`，
添加 `"onnx::AveragePool": "mindinsight.mindconverter.graph_based_converter.mapper.impl.nn.pool_mapper.PoolMapper"`来确定ONNX算子所对应的Mapper脚本文件。

## 验证自定义算子映射脚本

In [2]:
import numpy as np
from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper
from mindinsight.mindconverter.graph_based_converter.common.code_fragment import NewFragment


def test_mapper(onnx_info):
    """
    Test mapper.

    Args:
        onnx_info (dict): Onnx operator_info. Struct is
                                   {
                                    'op_name': op_name,
                                    'attributes': dict(),
                                    'weights': [NodeWeight(), ...]
                                   }
    """

    template, exchange_msg, outputs_lists, outputs_mapping = \
        ONNXToMindSporeMapper.convert(onnx_info['op_name'],
                                      onnx_info['attributes'],
                                      onnx_info['weights'])

    exchange_msg['var_0']['variable_name'] = 'self_defined_operator'
    exchange_msg['var_0']['inputs'] = ['x']

    fragment = NewFragment(data_entity=exchange_msg, code_template=template, outputs=outputs_lists,
                           outputs_mapping=outputs_mapping)

    code = fragment()
    init_code = code[0]
    construct_code = code[1]
    print('-'*30, 'init_code', '-'*30)
    print('\n'.join(init_code))
    print('-'*30, 'construct_code', '-'*30)
    print('\n'.join(construct_code))

In [3]:
onnx_operator_info = {'op_name': 'onnx::AveragePool',
                          'attributes': {'auto_pad': 'NOTSET',
                                         'ceil_mode': 0,
                                         'count_include_pad': 0,
                                         'kernel_shape': (5, 5),
                                         'pads': (0, 0, 0, 0),
                                         'strides': (2, 2),
                                         'input_shape': (1, 3, 224, 224),
                                         'output_shape': (1, 3, 112, 112)
                                        },
                          'weights': []}
test_mapper(onnx_operator_info)

------------------------------ init_code ------------------------------
self.pad_self_defined_operator = nn.Pad(paddings=((0, 0), (0, 0), (1, 2), (1, 2)))
self.self_defined_operator = nn.AvgPool2d(kernel_size=(5, 5), stride=(2, 2))
------------------------------ construct_code ------------------------------
opt_self_defined_operator = self.pad_self_defined_operator(x)
opt_self_defined_operator = self.self_defined_operator(opt_self_defined_operator)


## 权重迁移相关教程

以`onnx::Add`算子为例。

In [4]:
import numpy as np

from mindinsight.mindconverter.graph_based_converter.constant import ExchangeMessageKeywords, TemplateKeywords, \
    WeightType
from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper
from mindinsight.mindconverter.graph_based_converter.third_party_graph.onnx_utils import NodeWeight


class AddMapper(ONNXToMindSporeMapper):
    """Add mapper."""

    @staticmethod
    def _operation_name_in_ms(*args, **kwargs):
        return "P.Add"

    @staticmethod
    def _convert_params(**kwargs):
        return dict()

    @staticmethod
    def _convert_trained_weights(**kwargs):
        """
        权重迁移相关方法，返回数据体用于生成CheckPoint文件。

        Returns, dict(MindSpore算子权重名: {'data': 权重值, 'type': 权重类型， 'onnx_name': ONNX算子权重名})
        """
        weights = kwargs.get('weights', list())  # 获取算子输入当中的静态ensor数据体,即为该算子权重，保存在CheckPoint文件当中。
        tensor = AddMapper._find_val_by_index(0, weights)  # 获取权重值，类型为`numpy.ndarray`。
        onnx_name = AddMapper._find_onnx_name_by_index(0, weights)  # 获取权重在ONNX框架中的名称，主要用于权重共享相关功能。
        # 仅当静态tensor为`np.ndarray`且存在`shape`信息时，该tensor会被保存为权重。
        if isinstance(tensor, np.ndarray) and tensor.shape:
            return {'bias': {'data': tensor, 'type': WeightType.PARAMETER.value, 'onnx_name': onnx_name}}
        return dict()

    @staticmethod
    def _generate_snippet_template(**kwargs):
        template, exchange_msg, outputs_list, outputs_mapping = ONNXToMindSporeMapper._generate_snippet_template(
            **kwargs)
        op = kwargs.get("operation")
        args = kwargs.get("converted_params")
        weights = kwargs.get("weights")
        trainable_params = kwargs.get('trainable_params', dict())  # 获取`_convert_trained_weights`方法的返回值。
        if not op:
            raise ValueError("Can not get MindSpore operation name.")
        if not weights:
            return template, exchange_msg, outputs_list, outputs_mapping

        tensor = AddMapper._find_val_by_index(0, weights)
        bias_shape = tensor.shape
        # 该静态Tensor在原ONNX算子中的输入中的位置序列号，例如：在算子`onnx::Add(x, y)`中，`x`的位置序列号为0，`y`的位置序列号为1。
        bias_location = AddMapper._find_location_by_index(0, weights)

        variable_slot = "var_0"
        init_template = f"self.{{{variable_slot}}} = {op}({', '.join(['%s={%s}' % (p, p) for p in args])})"
        inputs_in_construct = [f"{{{ExchangeMessageKeywords.VariableScope.value.INPUTS.value}}}"]

        # 使用该位置序列号信息，确保该静态Tensor在生成的MindSpore算子中的输入顺序和原ONNX算子中的输入顺序保持一致。
        if bias_location != -1:
            inputs_in_construct.insert(bias_location, f"self.{{{variable_slot}}}_bias")

        # 构建出常量Tensor算子，作为算子的输入。
        # `XXX/bias`和`XXX_bias`当中的`bias`需要
        # 和`_convert_trained_weights`方法返回值当中定义的`bias`（MindSpore算子权重名）保持一致。
        if bias_shape:
            # Note: adding weight shape to args is now deprecated due to conflict of partial weights share processing.
            variable_slot_param_name = f"{variable_slot}/bias"  # XX/bias`
            init_tensor = f"self.{{{variable_slot}}}_bias = {{{variable_slot_param_name}}}"

        else:
            # 当`shape`信息为None时，`tensor.tolist()`返回单个数值，这种情况下，该值作为算子参数，构建出常量算子作为算子输入。
            args["bias_value"] = tensor.tolist()
            init_tensor = f"self.{{{variable_slot}}}_bias = {{bias_value}}"

        construct_template = f"opt_{{{variable_slot}}} = self.{{{variable_slot}}}" \
                             f"({', '.join(inputs_in_construct)})"
        template = {
            variable_slot: {
                TemplateKeywords.INIT.value: [init_template, init_tensor],
                TemplateKeywords.CONSTRUCT.value: [construct_template]
            }
        }
        exchange_msg = {
            variable_slot: {
                ExchangeMessageKeywords.VariableScope.value.OPERATION.value: op,
                ExchangeMessageKeywords.VariableScope.value.VARIABLE_NAME.value: None,
                ExchangeMessageKeywords.VariableScope.value.OUTPUT_TYPE.value:
                    ExchangeMessageKeywords.VariableScope.value.TSR_TYPE.value,
                ExchangeMessageKeywords.VariableScope.value.INPUTS.value: [],
                ExchangeMessageKeywords.VariableScope.value.ARGS.value: args,
                ExchangeMessageKeywords.VariableScope.value.WEIGHTS.value: weights,
                ExchangeMessageKeywords.VariableScope.value.TRAINABLE_PARAMS.value: trainable_params
            }
        }

        # 权重共享相关。声明权重名称，权重值由后续模块添加。
        if bias_shape:
            exchange_msg[variable_slot][ExchangeMessageKeywords.VariableScope.value.PARAMETERS_DECLARED.value] = {
                "bias": ""
            }
        outputs_list = [f"opt_{{{variable_slot}}}"]
        outputs_mapping = ((0, 0),)
        return template, exchange_msg, outputs_list, outputs_mapping

## 验证权重迁移算子映射脚本

In [5]:
import numpy as np
from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper
from mindinsight.mindconverter.graph_based_converter.common.code_fragment import NewFragment


def test_mapper(onnx_info):
    """
    Test mapper.

    Args:
        onnx_info (dict): Onnx operator_info. Struct is
                                   {
                                    'op_name': op_name,
                                    'attributes': dict(),
                                    'weights': [NodeWeight(), ...]
                                   }
    """

    template, exchange_msg, outputs_lists, outputs_mapping = \
        ONNXToMindSporeMapper.convert(onnx_info['op_name'],
                                      onnx_info['attributes'],
                                      onnx_info['weights'])

    exchange_msg['var_0']['variable_name'] = 'self_defined_operator'
    exchange_msg['var_0']['inputs'] = ['x']
    
    trainable_params = exchange_msg['var_0']['trainable_params']
    for weight_name, weight_inst in trainable_params.items():
        weight = weight_inst['data']
        weight_shape = weight.shape
        weight_dtype = weight.dtype
        exchange_msg['var_0']['parameters'][weight_name] = NewFragment.create_parameter(weight_shape, weight_dtype)

    fragment = NewFragment(data_entity=exchange_msg, code_template=template, outputs=outputs_lists,
                           outputs_mapping=outputs_mapping)

    code = fragment()
    init_code = code[0]
    construct_code = code[1]
    print('-'*30, 'init_code', '-'*30)
    print('\n'.join(init_code))
    print('-'*30, 'construct_code', '-'*30)
    print('\n'.join(construct_code))

In [6]:
onnx_operator_info = {'op_name': 'onnx::Add',
                          'attributes': {},
                          'weights': [NodeWeight(weight_name='onnx_bias',
                                                 weight_value=np.ones((1, 3, 224, 224), dtype=np.int),
                                                 weight_location=1)]}
test_mapper(onnx_operator_info)

------------------------------ init_code ------------------------------
self.self_defined_operator = P.Add()
self.self_defined_operator_bias = Parameter(Tensor(np.random.uniform(0, 1, (1, 3, 224, 224)).astype(np.int64)), name=None)
------------------------------ construct_code ------------------------------
opt_self_defined_operator = self.self_defined_operator(x, self.self_defined_operator_bias)
